In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

from mpmath import besseljzero
from scipy.special import gamma
from scipy.special import jv, iv, ive
from scipy.optimize import differential_evolution
from scipy.interpolate import interp1d

from scipy.stats import pearsonr
from sklearn.metrics import r2_score

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def simulate_HSDM_4D(a, mu, ndt, sigma=1, dt=0.001):
    x = np.zeros(mu.shape)
    
    rt = 0
    
    while np.linalg.norm(x, 2) < a(rt):
        x += mu*dt + sigma*np.sqrt(dt)*np.random.normal(0, 1, mu.shape)
        rt += dt
    
    theta1 = np.arctan2(np.sqrt(x[3]**2 + x[2]**2 + x[1]**2), x[0])
    theta2 = np.arctan2(np.sqrt(x[3]**2 + x[2]**2), x[1])
    theta3 = np.arctan2(x[3], x[2])
    
    
    return rt+ndt, (theta1, theta2, theta3)

In [3]:
def k(a, da, t, q, sigma=2):
    return 0.5 * (q - 0.5*sigma - da(t))

def psi(a, da, t, z, tau, q, sigma=2):
    kk = k(a, da, t, q, sigma)
    
    if 2*np.sqrt(a(t)*z)/(sigma*(t-tau))<=700:
        term1 = 1./(sigma*(t - tau)) * np.exp(- (a(t) + z)/(sigma*(t-tau)))
        term2 = (a(t)/z)**(0.5*(q-sigma)/sigma)
        term3 = da(t) - (a(t)/(t-tau)) + kk
        term4 = iv(q/sigma-1, 2*np.sqrt(a(t)*z)/(sigma*(t-tau)))
        term5 = (np.sqrt(a(t)*z)/(t-tau)) * iv(q/sigma, 2*np.sqrt(a(t)*z)/(sigma*(t-tau)))
    else:
        term1 = 1./(sigma*(t - tau))
        term2 = (a(t)/z)**(0.5*(q-sigma)/sigma)
        term3 = da(t) - (a(t)/(t-tau)) + kk
        term4 = ive(q/sigma-1, (a(t) + z)/(sigma*(t-tau)))
        term5 = (np.sqrt(a(t)*z)/(t-tau)) * ive(q/sigma, (a(t) + z)/(sigma*(t-tau)))
    
    return term1 * term2 * (term3 * term4 + term5)

def ie_bessel_fpt(a, da, q, z, sigma=2, dt=0.1, T_max=2):
    g = [0]
    T = [0]
    g.append(-2*psi(a, da, dt, z, 0, q, sigma))
    T.append(dt)
    
    for n in range(2, int(T_max/dt)+2):
        s = -2 * psi(a, da, n*dt, z, 0, q, sigma)

        for j in range(1, n):
            s += 2 * dt * g[j] * psi(a, da, n*dt, a(j*dt), j*dt, q, sigma)

        g.append(s)
        T.append(n*dt)
        
    g = np.asarray(g)
    T = np.asarray(T)
    
    gt = interp1d(T, g)
    return gt

In [4]:
def HSDM_4D_likelihood(prms, RT, Theta):
    a = lambda t: prms[0]**2
    da = lambda t: 0
    ndt = prms[1]
    mu = np.array([prms[2], prms[3], prms[4], prms[5]])
    
    fpt = ie_bessel_fpt(a, da, mu.shape[0], 0.0001, 
                        dt=0.1, T_max=max(RT))
    log_lik = 0
    for i in range(len(RT)):
        rt, theta = RT[i], Theta[i]
        if rt - ndt > 0.001:
            mu_dot_x0 = mu[0]*np.cos(theta[0])
            mu_dot_x1 = mu[1]*np.sin(theta[0])*np.cos(theta[1]) 
            mu_dot_x2 = mu[2]*np.sin(theta[0])*np.sin(theta[1])*np.cos(theta[2])
            mu_dot_x3 = mu[3]*np.sin(theta[0])*np.sin(theta[1])*np.sin(theta[2]) 
            term1 = prms[0] * (mu_dot_x0 + mu_dot_x1 + mu_dot_x2 + mu_dot_x3)
            term2 = 0.5 * np.linalg.norm(mu, 2)**2 * (rt-ndt)
            
            density = np.exp(term1 - term2) * fpt(rt - ndt)
            
            if 0.1**14 < density:
                log_lik += -np.log(density)
            else:
                log_lik += -np.log(0.1**14)
        else:
            log_lik += -np.log(0.1**14)
        
    return log_lik

In [5]:
recovery_df = {'threshold_true': [],
               'threshold_estimate': [],
               'ndt_true': [],
               'ndt_estimate': [],
               'mu1_true': [],
               'mu1_estimate': [],
               'mu2_true': [],
               'mu2_estimate': [],
               'mu3_true': [],
               'mu3_estimate': [],
               'mu4_true': [],
               'mu4_estimate': []}

min_threshold = 0.5
max_threshold = 6

min_ndt = 0.1
max_ndt = 1

min_mu = -5
max_mu = 5

In [6]:
for n in tqdm(range(150)):
    threshold = np.random.uniform(min_threshold, max_threshold)
    a = lambda t: threshold
    ndt = np.random.uniform(min_ndt, max_ndt)
    mu = np.array([np.random.uniform(min_mu, max_mu), 
                   np.random.uniform(min_mu, max_mu),
                   np.random.uniform(min_mu, max_mu),
                   np.random.uniform(min_mu, max_mu)])
    
    
    recovery_df['threshold_true'].append(threshold)
    recovery_df['ndt_true'].append(ndt)
    recovery_df['mu1_true'].append(mu[0])
    recovery_df['mu2_true'].append(mu[1])
    recovery_df['mu3_true'].append(mu[2])
    recovery_df['mu4_true'].append(mu[3])
    
    RT = []
    Theta = []
    
    for i in range(250):
        rt, theta = simulate_HSDM_4D(a, mu, ndt)
        RT.append(rt)
        Theta.append(theta)
    
    min_ans = differential_evolution(HSDM_4D_likelihood,
                                     args=(RT, Theta),
                                     bounds=[(min_threshold, max_threshold), (min_ndt, max_ndt),
                                             (min_mu, max_mu), (min_mu, max_mu), 
                                             (min_mu, max_mu), (min_mu, max_mu)])
    
    recovery_df['threshold_estimate'].append(min_ans.x[0])
    recovery_df['ndt_estimate'].append(min_ans.x[1])
    recovery_df['mu1_estimate'].append(min_ans.x[2])
    recovery_df['mu2_estimate'].append(min_ans.x[3])
    recovery_df['mu3_estimate'].append(min_ans.x[4])
    recovery_df['mu4_estimate'].append(min_ans.x[5])

    
recovery_df = pd.DataFrame(recovery_df)

100%|█████████████████████████████████████████| 150/150 [34:37<00:00, 13.85s/it]


In [7]:
recovery_df

,threshold_true,threshold_estimate,ndt_true,ndt_estimate,mu1_true,mu1_estimate,mu2_true,mu2_estimate,mu3_true,mu3_estimate,mu4_true,mu4_estimate
0,2.463319,2.643005,0.552864,0.581073,-1.022397,-1.249622,-0.119238,-0.002543,-3.751641,-4.269524,-4.740251,-5.000000
1,4.318612,4.461053,0.868799,0.843171,3.361790,3.341421,0.071290,0.059838,-1.351724,-1.312049,2.065352,2.023889
2,1.080623,1.078395,0.539248,0.595496,3.082894,4.940471,0.049351,0.004880,1.026532,1.369438,-4.008273,-4.946153
3,1.738533,1.698411,0.639589,0.671672,0.446921,0.255269,-0.722997,-0.971378,-0.096092,0.003056,-3.082051,-3.324308
4,2.513461,2.643790,0.405046,0.401189,-2.134385,-2.186347,-1.627681,-1.706214,-0.526245,-0.535135,1.888072,2.079372
...,...,...,...,...,...,...,...,...,...,...,...,...
145,3.732090,4.266302,0.928434,0.912889,-3.274884,-3.614484,4.861707,5.000000,0.980098,0.905111,1.735979,1.908552
146,2.607018,2.792880,0.827472,0.871143,2.237078,2.767911,0.906682,1.013014,4.386928,5.000000,2.378040,2.913703
147,5.692206,5.605272,0.677825,0.787351,-1.455309,-1.605681,-0.884924,-1.026893,-0.514658,-0.509969,3.394705,3.601969
148,2.645942,2.334835,0.589384,0.681972,-4.620156,-5.000000,-0.365595,-0.633431,4.951730,5.000000,-0.152311,-0.164064


In [8]:
file_name = 'IE_4d_recovery_1.csv'
old_recovery_data = pd.read_csv(file_name, index_col=0)
recovery_df = pd.concat([old_recovery_data, 
                         recovery_df]).reset_index(drop=True)
recovery_df.to_csv(file_name)